理论认为距离消费终端越近，价格的传导就越慢，而白条和生猪的价格以及之间的差价反应了猪肉行业“增加值”在不同产业链的分布情况，也反映了中间环节在传递供需双方信息时的倾向。从数据分类中有探讨到毛白价差的变化引起来源不同，代表了市场中供需的不同变化，而不同的供需变化又会影响到后面的价格。那么毛白价差的变化是否会影响到后期猪价变化，用格兰杰因果检验测试：

第一步：进行ADF检验和协整检验，如果都不通过则进行差分，以便后面进行VAR确定最优滞后阶数。论认为距离消费终端越近，价格的传导就越慢，而白条和生猪的价格以及之间的差价反应了猪肉行业“增加值”在不同产业链的分布情况，也反映了中间环节在传递供需双方信息时的倾向。从数据分类中有探讨到毛白价差的变化引起来源不同，代表了市场中供需的不同变化，而不同的供需变化又会影响到后面的价格。那么毛白价差的变化是否会影响到后期猪价变化，用格兰杰因果检验测试：

第一步：进行ADF检验和协整检验，如果都不通过则进行差分，以便后面进行VAR确定最优滞后阶数。

In [2]:
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
import pandas as pd
import numpy as np

In [3]:
# 引入Elite日度毛白价差和金猪日度生猪均价
dataPigPrice = pd.read_excel(r"D:\git\新建文件夹\swine-data-analysis\data_clear_and_analysis\database\GL_20230227.xlsx", sheet_name="出栏价格")
dataPigPriceAndBarPrice = pd.read_excel(r"database/ELite_20230320.xlsx", sheet_name="白毛价差")

In [4]:
# 取生猪价格和毛白价差连接，并更改列名
dataPigPrice = dataPigPrice.iloc[:, 1:3]
dataPigPrice = dataPigPrice.drop(index = dataPigPrice.index[[0,1]])
dataPigPriceAndBarPrice = dataPigPriceAndBarPrice.drop(index = dataPigPriceAndBarPrice.index[[0]])

In [5]:
dataPigPrice.columns = ['Time', 'PigPrice']
dataPigPriceAndBarPrice.columns = ['Time', '毛白价差']
dataPigPrice['PigPrice'].fillna(method='bfill', inplace=True)

In [6]:
dataPigPriceAndBarPrice['毛白价差'].fillna(method='bfill', inplace=True)

In [7]:
dataUse = dataPigPriceAndBarPrice.set_index('Time').join(dataPigPrice.set_index('Time'))
dataUseNoNan = dataUse.dropna()

In [8]:
dataUseNoNan

,毛白价差,PigPrice
Time,,
2023-03-10,3.95,15.57
2023-03-09,3.96,15.57
2023-03-08,3.89,15.75
2023-03-07,3.80,15.83
2023-03-06,3.85,15.68
...,...,...
2018-01-08,5.18,15.34
2018-01-05,5.23,15.12
2018-01-04,5.23,15.06


In [9]:
print(adfuller(dataUseNoNan['PigPrice']))
print(adfuller(dataUseNoNan['毛白价差']))

(-1.6236178509391657, 0.4708331120564497, 9, 1283, {'1%': -3.435457117423131, '5%': -2.863795358068796, '10%': -2.567970771161219}, 869.9049156290052)
(-1.9750178752123888, 0.29763315318814504, 0, 1292, {'1%': -3.4354214704688113, '5%': -2.863779629316101, '10%': -2.567962394851863}, 55.056868374694204)


从生猪价格的adf检验结果来说不能拒绝原假设，所以为非平稳序列，需要进行差分后得到平稳序列

In [10]:
# 将生猪价格和毛白价差进行差分
pigPrice_transform1 = dataUseNoNan['PigPrice'].diff().dropna()
pigPriceAndBar_transform1 = dataUseNoNan['毛白价差'].diff().dropna()

In [11]:
print(adfuller(pigPrice_transform1))
print(adfuller(pigPriceAndBar_transform1))

(-8.298303351711777, 4.0829585649289577e-13, 8, 1283, {'1%': -3.435457117423131, '5%': -2.863795358068796, '10%': -2.567970771161219}, 870.913301365018)
(-36.44413153722199, 0.0, 0, 1291, {'1%': -3.4354254066484664, '5%': -2.863781366113082, '10%': -2.56796331977745}, 59.53633596673217)


经过1阶差分后两个序列在adf检验下呈平稳状态，可以进行VAR寻找最优滞后阶数
VAR：

In [12]:
dataTransformed = pd.DataFrame({'pigPrice':pigPrice_transform1, 'pigPriceAndBar':pigPriceAndBar_transform1})

In [16]:
model = VAR(dataTransformed)
resultVARAic = {}
for i in range(16):
    result = model.fit(i)
    print('Lag Order =', i)
    print('AIC = ', result.aic)
    resultVARAic[i] = result.aic

Lag Order = 0
AIC =  -4.86297385891561
Lag Order = 1
AIC =  -5.031804247367507
Lag Order = 2
AIC =  -5.040153858253938
Lag Order = 3
AIC =  -5.052833563410093
Lag Order = 4
AIC =  -5.055180651660745
Lag Order = 5
AIC =  -5.063894592928211
Lag Order = 6
AIC =  -5.0606402769494
Lag Order = 7
AIC =  -5.056568333604771
Lag Order = 8
AIC =  -5.056144327210202
Lag Order = 9
AIC =  -5.051849667666991
Lag Order = 10
AIC =  -5.050694291078923
Lag Order = 11
AIC =  -5.045364090296236
Lag Order = 12
AIC =  -5.042335985852025
Lag Order = 13
AIC =  -5.04104303730615
Lag Order = 14
AIC =  -5.038788200808069
Lag Order = 15
AIC =  -5.043265913217724


d:\miniconda\envs\ckywork\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
d:\miniconda\envs\ckywork\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [17]:
resultVARAic

{0: -4.86297385891561,
 1: -5.031804247367507,
 2: -5.040153858253938,
 3: -5.052833563410093,
 4: -5.055180651660745,
 5: -5.063894592928211,
 6: -5.0606402769494,
 7: -5.056568333604771,
 8: -5.056144327210202,
 9: -5.051849667666991,
 10: -5.050694291078923,
 11: -5.045364090296236,
 12: -5.042335985852025,
 13: -5.04104303730615,
 14: -5.038788200808069,
 15: -5.043265913217724}

In [20]:
min(resultVARAic.items(), key=lambda x:x[1])[0]

5

从上文可以得到最优滞后阶数为5，将阶数带入到格兰杰因果检验中查看

In [21]:
maxlag=5
test = 'ssr_chi2test'

def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    
   
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df

grangers_causation_matrix(dataTransformed, variables = dataTransformed.columns)


,pigPrice_x,pigPriceAndBar_x
pigPrice_y,1.0,0.0
pigPriceAndBar_y,0.0,1.0


In [22]:
grangercausalitytests(dataTransformed[['pigPrice', 'pigPriceAndBar']], maxlag=5)


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=43.3045 , p=0.0000  , df_denom=1288, df_num=1
ssr based chi2 test:   chi2=43.4053 , p=0.0000  , df=1
likelihood ratio test: chi2=42.6916 , p=0.0000  , df=1
parameter F test:         F=43.3045 , p=0.0000  , df_denom=1288, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=22.3800 , p=0.0000  , df_denom=1285, df_num=2
ssr based chi2 test:   chi2=44.9343 , p=0.0000  , df=2
likelihood ratio test: chi2=44.1694 , p=0.0000  , df=2
parameter F test:         F=22.3800 , p=0.0000  , df_denom=1285, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=20.5939 , p=0.0000  , df_denom=1282, df_num=3
ssr based chi2 test:   chi2=62.1191 , p=0.0000  , df=3
likelihood ratio test: chi2=60.6687 , p=0.0000  , df=3
parameter F test:         F=20.5939 , p=0.0000  , df_denom=1282, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=15.8300 , p=0.

{1: ({'ssr_ftest': (43.30447073488469, 6.799123007640671e-11, 1288.0, 1),
   'ssr_chi2test': (43.40533518535414, 4.449732625446399e-11, 1),
   'lrtest': (42.69161287194038, 6.408648123615165e-11, 1),
   'params_ftest': (43.30447073488432, 6.799123007642362e-11, 1288.0, 1.0)},
   array([[0., 1., 0.]])]),
 2: ({'ssr_ftest': (22.38004780099529, 2.792008425141405e-10, 1285.0, 2),
   'ssr_chi2test': (44.93425939810728, 1.7484352251594674e-10, 2),
   'lrtest': (44.16937859673635, 2.5629569014422325e-10, 2),
   'params_ftest': (22.380047800995246, 2.792008425141405e-10, 1285.0, 2.0)},
   array([[0., 0., 1., 0., 0.],
          [0., 0., 0., 1., 0.]])]),
 3: ({'ssr_ftest': (20.593918333299854, 4.917692785449507e-13, 1282.0, 3),
   'ssr_chi2test': (62.11909687587406, 2.0720105320012305e-13, 3),
   'lrtest': (60.66870012397385, 4.230297279047484e-13, 3),
   'params_ftest': (20.59391833329988, 4.917692785448773e-13, 1282.0, 3.0)},
   array([[0., 0., 0., 1., 0., 0., 0.],
          [0., 0., 0., 0., 1

从一阶差分结果的格兰杰分析来看，滞后阶数为5阶的毛白价差对猪价是有影响的，而从理论本身来看。